# Importing Libraries and Dataset

In [1]:
# importing libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
from sklearn.preprocessing import StandardScaler

from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [7]:
# importing datasets
df = pd.read_excel('Online Retail.xlsx')
df.head

<bound method NDFrame.head of      shrimp,almonds,avocado,vegetables mix,green grapes,whole weat flour,yams,cottage cheese,energy drink,tomato juice,low fat yogurt,green tea,honey,salad,mineral water,salmon,antioxydant juice,frozen smoothie,spinach,olive oil
0                                burgers,meatballs,eggs                                                                                                                                                                             
1                                               chutney                                                                                                                                                                             
2                                        turkey,avocado                                                                                                                                                                             
3     mineral water,milk,energy bar,whole wheat rice..

# Analysis

In [6]:
# checking for the records by country
df.Country.value_counts()

AttributeError: 'DataFrame' object has no attribute 'Country'

In [8]:
# based on above observations, taking out the Germany values
df = df[df['Country'] == 'Germany']

KeyError: 'Country'

In [ ]:
# understanding dataset
df.info()

In [ ]:
# checking for null values
df.isnull().sum()

In [ ]:
# looking for stats
df.describe()

In [ ]:
# checking the samples for any discripency
df.InvoiceNo.sample(10)

In [ ]:
# heccking the records for negative values
print('shape of data containg quantity less than 0:', df[df.Quantity <=0].shape)
print('shape of data containg unitprice is negative:', df[df.UnitPrice <=0].shape)

In [ ]:
# removing the above observations from the dataset
df = df[(df.Quantity > 0) & (df.UnitPrice > 0)]
df.shape

# Market Business Analysis

In [ ]:
# filtering the dataset
df_mba = df[['InvoiceNo', 'Description', 'Quantity']]
df_mba.head(2)

In [ ]:
# Total unique items present in final dataset
len(df_mba.Description.unique())

In [ ]:
# Total unique InvoiceNo present in final dataset
len(df_mba.InvoiceNo.unique())

In [ ]:
# printing the top 20 selling items

top_20 = pd.DataFrame(df_mba.Description.value_counts().head(20))
top_20.reset_index(inplace=True)
top_20.columns = ['Description', 'Count']

In [ ]:
top_20.head(3)

In [ ]:
# plotting the count of each item
sns.barplot(top_20, y="Description", x="Count", orient = 'h');

In [ ]:
# Grouping the Invoices based on the description of the items
df_mba = df_mba.groupby('InvoiceNo')['Description'].agg(lambda x: ', '.join(x)).reset_index()
df_mba.head(2)

In [ ]:
# shape of the final dataset
df_mba.shape

In [ ]:
# Splitting the Description column
df_items = df_mba['Description'].str.split(', ', expand=True)

# one hot encoding
df_ohe = pd.get_dummies(df_items.apply(lambda x: x.str.strip()), prefix='', prefix_sep='').groupby(level=0, axis=1).max()

# final dataset
df_final = pd.concat([df_mba, df_ohe], axis=1)

In [ ]:
# dropping the Description column
df_final.drop('Description', axis =1, inplace =True)

# setting invoice no as index
df_final.set_index('InvoiceNo', inplace =True)

In [ ]:
df_final.head(2)

## Apriori Model

- min_support : float (default: 0.5)

A float between 0 and 1 for minumum support of the itemsets returned. The support is computed as the fraction transactions_where_item(s)_occur / total_transactions.

- use_colnames : bool (default: False)

If True, uses the DataFrames' column names in the returned DataFrame instead of column indices.

- max_len : int (default: None)

Maximum length of the itemsets generated. If None (default) all possible itemsets lengths (under the apriori condition) are evaluated.

- verbose : int (default: 0)

Shows the number of iterations if >= 1 and low_memory is True. If

=1 and low_memory is False, shows the number of combinations.

- low_memory : bool (default: False)

If True, uses an iterator to search for combinations above min_support. Note that while low_memory=True should only be used for large dataset if memory resources are limited, because this implementation is approx. 3-6x slower than the default.

#### Try 1

In [ ]:
# instanciating the apriori model with default parameters

df_support = apriori(df_final)

In [ ]:
# sorting the result based on the support, maximum on top
df_support = df_support.sort_values("support", ascending = False)

In [ ]:
# creating a column indicating number of items in each bucket
df_support['total Items'] = df_support['itemsets'].apply(lambda x: len(x))

In [ ]:
print(df_support.shape)
df_support.head(10)

By Using default parameters, we are not getting any output from the data.
So, trying with different random parameters.

#### Try 2

In [ ]:
# instanciating the apriori model

df_support1 = apriori(df_final, min_support = 0.1, use_colnames = True, verbose = 1)


In [ ]:
# sorting the result based on the support, maximum on top
df_support1 = df_support1.sort_values("support", ascending = False)

In [ ]:
# creating a column indicating number of items in each bucket
df_support1['total Items'] = df_support1['itemsets'].apply(lambda x: len(x))

In [ ]:
print(df_support1.shape)
df_support1.head(10)

In [ ]:
# printing out the association dataset, based on the confidence metric
df_ass_rule1 = association_rules(df_support1, metric="confidence", min_threshold=0.2)

# sorting the values based on confidence
df_ass_rule1 = df_ass_rule1.sort_values('confidence',ascending=False)

In [ ]:
print(df_ass_rule1.shape)
df_ass_rule1.head(10)

In [ ]:
top = df_ass_rule1.head(8)

top = [list(x) for x in top['antecedents'].tolist()]
top = [i[0] for i in top ]
top

**Observations**

Based on the above observation and Confidence values, we can say that following items are highly associated with the `Postage`.

['ROUND SNACK BOXES SET OF 4 FRUITS',
 'ROUND SNACK BOXES SET OF 4 FRUITS',
 'PLASTERS IN TIN SPACEBOY',
 'ROUND SNACK BOXES SET OF4 WOODLAND',
 'WOODLAND CHARLOTTE BAG',
 'REGENCY CAKESTAND 3 TIER',
 'PLASTERS IN TIN CIRCUS PARADE',
 'PLASTERS IN TIN WOODLAND ANIMALS']

- Antecedents:

The items or features that appear before the arrow in the association rule (A -> B). These are the conditions or items present in the dataset.

- Consequents:

The items or features that appear after the arrow in the association rule (A -> B). These are the outcomes or items that are predicted or inferred based on the antecedents.

- Antecedent Support:

The proportion of transactions in the dataset that contain the antecedent itemset. It gives an indication of how frequently the antecedent occurs.

- Consequent Support:

The proportion of transactions in the dataset that contain the consequent itemset. It gives an indication of how frequently the consequent occurs.

- Support:

The proportion of transactions in the dataset that contain both the antecedent and consequent itemsets. It is a measure of how frequently the rule is observed.

- Confidence:

The likelihood that the rule is true. It is calculated as the support of the itemset {A, B} divided by the support of A. For the rule A -> B, confidence indicates the probability of B occurring given that A has occurred.

- Lift:

Indicates the degree to which the occurrence of B is dependent on the occurrence of A, compared to their individual occurrences. A lift greater than 1 suggests a positive correlation, while a lift less than 1 suggests a negative correlation.

- Leverage:

Measures the difference between the observed support of the itemset {A, B} and the expected support if A and B were independent. Positive values indicate that the occurrence of A and B together is more frequent than expected.

- Conviction:

Measures the ratio of the expected frequency that A occurs without B to the observed frequency that A occurs without B. Higher conviction values indicate stronger dependency between A and B.

- Zhang's Metric:

An alternative metric for measuring the interestingness of association rules. It considers both confidence and support and is used to rank rules.
Interpreting the output involves analyzing these metrics to identify meaningful and actionable rules. For example, high confidence and lift values indicate strong associations, while low confidence might suggest weak or spurious relationships.

#RFM Model

In [ ]:
df.head(1)

In [ ]:
# converting customerId into integer format

df['CustomerID'] = df['CustomerID'].astype(int)

In [ ]:
# calculating the total spending

df['Total_cost'] = df['Quantity'] * df['UnitPrice']

In [ ]:
# filtering the dataset

df_rfm = df[['InvoiceNo',	'InvoiceDate', 'CustomerID', 'Total_cost']]
df_rfm.head(2)

In [ ]:
# calculating the amount of money spent by each customer

df_money = df_rfm.groupby('CustomerID')['Total_cost'].sum()
df_money = df_money.reset_index()
df_money.head()

In [ ]:
# calculating how frequently

df_freq = df_rfm.groupby(['CustomerID','InvoiceDate'])['InvoiceNo'].count()

df_freq = df_freq.reset_index()

df_freq = df_freq[['CustomerID', 'InvoiceNo']]

df_freq = df_freq.groupby('CustomerID')['InvoiceNo'].count()

df_freq = df_freq.reset_index()

df_freq.head()

In [ ]:
# identifying the minimum and maximum date present in dataset

df_rfm.InvoiceDate.min(), df_rfm.InvoiceDate.max()

In [ ]:
# Calculating the date diff, which indicates how recently customer has purchased the items

df_rfm['time_diff'] = df_rfm.InvoiceDate.max() - df_rfm['InvoiceDate']

In [ ]:
# Extracting the days in time_diff column

df_rfm['time_diff'] = df_rfm['time_diff'].dt.days

df_rfm.head(2)

In [ ]:
# Aggregating the customerID with respect to date diff column

df_recency = df_rfm.groupby('CustomerID')['time_diff'].min()
df_recency = df_recency.reset_index()
df_recency.head()

In [ ]:
# Creating the dataset with all 3 determined quantities

df_cluster = pd.merge(df_recency, df_freq, on = 'CustomerID', how = 'inner').merge(df_money, on = 'CustomerID', how = 'inner')
df_cluster.head()

In [ ]:
df_cluster.describe()

In [ ]:
# Checking the outliers

plt.figure(figsize = (7,3))
sns.boxplot(x=df_cluster['time_diff'])

In [ ]:
# removing the outliers from time_diff column

Q1 = df_cluster['time_diff'].quantile(0.05)
Q3 = df_cluster['time_diff'].quantile(0.60)
IQR = Q3 - Q1
df_cluster = df_cluster[(df_cluster['time_diff'] >= Q1 - 1.5*IQR) & (df_cluster['time_diff'] <= Q3 + 1.5*IQR)]

plt.figure(figsize = (7,3))
sns.boxplot(x=df_cluster['time_diff'])

In [ ]:
# checking for the outliers present in InvoiceNo column

plt.figure(figsize = (7,3))
sns.boxplot(x=df_cluster['InvoiceNo'])

In [ ]:
# Removing the outliers present in InvoiceNo column

Q1 = df_cluster['InvoiceNo'].quantile(0.05)
Q3 = df_cluster['InvoiceNo'].quantile(0.7)
IQR = Q3 - Q1
df_cluster = df_cluster[(df_cluster['InvoiceNo'] >= Q1 - 1.5*IQR) & (df_cluster['InvoiceNo'] <= Q3 + 1.5*IQR)]

plt.figure(figsize = (7,3))
sns.boxplot(x=df_cluster['InvoiceNo'])

In [ ]:
# checking the outliers present in Total_cost column

plt.figure(figsize = (7,3))
sns.boxplot(x=df_cluster['Total_cost'])

In [ ]:
# removing the outliers from Total_cost column

Q1 = df_cluster['Total_cost'].quantile(0.05)
Q3 = df_cluster['Total_cost'].quantile(0.7)
IQR = Q3 - Q1
df_cluster = df_cluster[(df_cluster['Total_cost'] >= Q1 - 1.5*IQR) & (df_cluster['Total_cost'] <= Q3 + 1.5*IQR)]

plt.figure(figsize = (7,3))
sns.boxplot(x=df_cluster['Total_cost'])

In [ ]:
# scaling

In [ ]:
# Standardizing the dataset for further analysis
sc = StandardScaler()
df_cluster_sc = sc.fit_transform(df_cluster[['time_diff'	,'InvoiceNo'	,'Total_cost']])

# converting into dataset
df_cluster_sc = pd.DataFrame(df_cluster_sc)
df_cluster_sc.columns = ['time_diff'	,'InvoiceNo'	,'Total_cost']
df_cluster_sc.head()

## Kmeans clustering

In [ ]:
# importing dataset

import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

In [ ]:
# elbow-curve/SSD
# calculating the sum of squared distance for optimum cluster values

ssd = []
range_n_clusters = [2, 3, 4, 5, 6, 7, 8]
for num_clusters in range_n_clusters:
    kmeans = KMeans(n_clusters=num_clusters, max_iter=50)
    kmeans.fit(df_cluster_sc)

    ssd.append(kmeans.inertia_)

# plot the SSDs for each n_clusters
# ssd
plt.plot(ssd);

In [ ]:
ssd

In [ ]:
# silhouette analysis
range_n_clusters = [2, 3, 4, 5, 6, 7, 8]

for num_clusters in range_n_clusters:

    # intialise kmeans
    kmeans = KMeans(n_clusters=num_clusters, max_iter=50)
    kmeans.fit(df_cluster_sc)

    cluster_labels = kmeans.labels_

    # silhouette score
    silhouette_avg = silhouette_score(df_cluster_sc, cluster_labels)
    print("For n_clusters={0}, the silhouette score is {1}".format(num_clusters, silhouette_avg))


In [ ]:
# final model with k=3
kmeans = KMeans(n_clusters=3, max_iter=50)
kmeans.fit(df_cluster_sc)

In [ ]:
# assign the label
df_cluster['cluster_id'] = kmeans.labels_
df_cluster.head()

In [ ]:
# plot
sns.boxplot(x='cluster_id', y='Total_cost', data=df_cluster,)

In [ ]:
# plot
sns.boxplot(x='cluster_id', y='InvoiceNo', data=df_cluster)

In [ ]:

# plot
sns.boxplot(x='cluster_id', y='time_diff', data=df_cluster)